<a> <img src="Toronto-skyline.jpg" width=600 align="center"></a>

<h1 align="left"><font size="5">Toronto Neighborhoods - Segmenting and Clustering</font></h1>

### Table of Contents

1. Import python libraries, then import and wrangle the Toronto neighborhood data
2. __Toronto Dataframe__
3. __Toronto Dataframe with Latitude and Longitude__
4. Explore the neighborhoods in Toronto
5. Analyze Each Neighborhood
6. Cluster Neighborhoods
7. __Map and Examination of Clusters__

### 1. Import libraries

In [1]:
# Import libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if necessary
# get_ipython().system("conda install -c conda-forge geopy --yes") # uncomment this line necessary
# !pip install geopy  # this is the option that worked. Only needs to be run once

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
# from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe -- deprecated
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Additional libraries to import
# !pip install geocoder
import geocoder
from IPython.display import display

print('Libraries imported.')

Libraries imported.


#### Load the table of "M" postal codes of Canada from the wiki page

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df_orig = pd.read_html(url)[0]

#### Remove rows that have "Not assigned" in the Borough column

In [3]:
df = df_orig.copy()  # create a deep copy
df.drop(df[df.Borough == "Not assigned"].index, inplace=True)
df.reset_index(drop=True, inplace=True)
# df.head()

#### Replace multiple neighborhoods with a comma-separated list

In [4]:
df["Neighborhood"] = df.Neighborhood.replace({" /":","}, regex=True)
# df.head()

#### Look for any "Not assigned" neighborhoods, and replace "Not assigned" with the borough name

In [5]:
# Note: there is almost certainly a one-line version of the below loop, but this works
for idx in df.index:
    if df["Neighborhood"][idx] == "Not assigned":
        df["Neighborhood"][idx] = df["Borough"][idx]

## 2. Toronto Dataframe (first 10 rows)

In [6]:
df.head(10)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Show the number of rows in the dataframe

In [7]:
print(f"The Toronto neighborhood dataframe has {df.shape[0]} rows.")
print('The dataframe has {} "M" postal codes and {} boroughs. There are {} rows in the dataframe.'.format(
        len(df['Postal code'].unique()),len(df['Borough'].unique()),df.shape[0]    )
)

The Toronto neighborhood dataframe has 103 rows.
The dataframe has 103 "M" postal codes and 10 boroughs. There are 103 rows in the dataframe.


#### Get the latitude and longitude for each postal code

In [8]:
df=df.reindex(columns=[*df.columns.tolist(), 'Latitude', 'Longitude'], fill_value=None)

In [9]:
for idx in df.index:
    postalCode =  df["Postal code"][idx]
    lat_lng_coords = None

    # loop until we get the coordinates, with a max of 10 tries per postal code
    maxcount = count = 0
    while(lat_lng_coords is None):
        g = geocoder.arcgis(f'{postalCode}, Toronto, Ontario')  # use "arcgis" instead of "google"
        lat_lng_coords = g.latlng
        count += 1
        if count > 10:
            lat_lng_coords = [-999,-999]
        maxcount = max(count,maxcount)
        
    df.loc[idx,"Latitude"]  = lat_lng_coords[0]
    df.loc[idx,"Longitude"] = lat_lng_coords[1]
    print('.',end="")  # geocoder calls are slow, so this shows there is progress
    if not (idx+1)%20: print("")

print("\n")
# print(f"Most iterations required for a postal code: {maxcount}.")

....................
....................
....................
....................
....................
...



## 3. Dataframe with added Latitude and Longitude:

In [10]:
df.head(10)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
5,M9A,Etobicoke,Islington Avenue,43.667481,-79.528953
6,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
7,M3B,North York,Don Mills,43.748900,-79.357220
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.707193,-79.311529
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529


#### Create a map of Toronto with neighborhoods superimposed on top.

In [11]:
# create map of Toronto using latitude and longitude values
g = geocoder.arcgis(f'Toronto, Ontario')  # use "arcgis" instead of "google"
latitude, longitude = g.latlng

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, postalCode, borough in zip(df['Latitude'], df['Longitude'], df['Postal code'], df['Borough']):
    label = '{}, {}'.format(postalCode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define the URL, with credentials

In [12]:
# CLIENT_ID = 'your-client-ID' # your Foursquare ID
# CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
# VERSION = '20180605' # Foursquare API version
CLIENT_ID     = 'HYJIVB4VRZEEUO5SXGXZ2LVNLBQ0WTDUZXL3PEO3JKJIDBB1' # your Foursquare ID
CLIENT_SECRET = 'PYPFOFOGOGFMMOFKZ2QU0RAJ1I4WHW3CWWMBL3GMBZFRYAOP' # your Foursquare Secret
VERSION       = '20180604'
LIMIT = 30

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: HYJIVB4VRZEEUO5SXGXZ2LVNLBQ0WTDUZXL3PEO3JKJIDBB1
CLIENT_SECRET:PYPFOFOGOGFMMOFKZ2QU0RAJ1I4WHW3CWWMBL3GMBZFRYAOP


In [14]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=HYJIVB4VRZEEUO5SXGXZ2LVNLBQ0WTDUZXL3PEO3JKJIDBB1&client_secret=PYPFOFOGOGFMMOFKZ2QU0RAJ1I4WHW3CWWMBL3GMBZFRYAOP&v=20180604&ll=43.648690000000045,-79.38543999999996&radius=500&limit=100'

Send the GET request and examine the results

In [15]:
results = requests.get(url).json()

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

# nearby_venues.head()

In [18]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## 4. Explore Neighborhoods in Toronto

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(f'{name}, lat={lat}, lng={lng}')
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
toronto_venues = getNearbyVenues(names=df['Postal code'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

#### Let's check the size of the resulting dataframe

In [22]:
print(toronto_venues.shape)
toronto_venues.head()

(2365, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.752935,-79.335641,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.752935,-79.335641,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,43.728102,-79.311890,Tim Hortons,43.725517,-79.313103,Coffee Shop
3,M4A,43.728102,-79.311890,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,M4A,43.728102,-79.311890,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection


Let's check how many venues were returned for each neighborhood

In [23]:
# toronto_venues.groupby('Neighborhood').count()

#### Let's find out how many unique categories can be curated from all the returned venues

In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 267 uniques categories.


## 5. Analyze Each Neighborhood

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# toronto_onehot.head()

In [26]:
toronto_onehot.shape

(2365, 267)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
# toronto_grouped

#### Let's confirm the new size

In [28]:
toronto_grouped.shape

(100, 267)

#### Print each neighborhood along with the top 5 most common venues for the first 3 neighborhoods

In [29]:
num_top_venues = 5
count = 0
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    count += 1
    if count >= 3:
        break

----M1B----
          venue  freq
0         Trail   1.0
1   Yoga Studio   0.0
2        Museum   0.0
3  Noodle House   0.0
4     Nightclub   0.0


----M1E----
                    venue  freq
0             Pizza Place  0.10
1    Fast Food Restaurant  0.10
2      Mexican Restaurant  0.05
3  Thrift / Vintage Store  0.05
4              Sports Bar  0.05


----M1G----
                     venue  freq
0              Coffee Shop  0.67
1        Korean Restaurant  0.33
2  New American Restaurant  0.00
3                   Office  0.00
4  North Indian Restaurant  0.00




#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

# neighborhoods_venues_sorted.head()

## 6. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [32]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 4, 0, 4, 4, 4, 2, 4, 4], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [33]:
# neighborhoods_venues_sorted.columns

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [35]:
# neighborhoods_venues_sorted.head()

In [36]:
toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Postal code')


toronto_merged.dropna(subset=['Cluster Labels'],inplace=True)  # drop any rows with NaN for the Cluster Label
toronto_merged.reset_index(drop=True, inplace=True)            # reset index
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype(int) # change Cluster Label column to type "int"

toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752935,-79.335641,1,Food & Drink Shop,Park,Farm,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Donut Shop
1,M4A,North York,Victoria Village,43.728102,-79.311890,4,Park,Pizza Place,Portuguese Restaurant,Intersection,Coffee Shop,Farm,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,4,Coffee Shop,Pub,Café,Music Venue,Athletics & Sports,Performing Arts Venue,Theater,Mexican Restaurant,Seafood Restaurant,Food Truck
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211,4,Clothing Store,Bookstore,Sushi Restaurant,Men's Store,Food Court,Cosmetics Shop,American Restaurant,Furniture / Home Store,Seafood Restaurant,Café
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,4,Coffee Shop,Café,Yoga Studio,Sandwich Place,Italian Restaurant,Juice Bar,Bookstore,Fried Chicken Joint,Burger Joint,Burrito Place


Finally, let's visualize the resulting clusters

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

## 7. Map and Examination of Clusters

In [38]:
map_clusters

### Finally, examine each cluster and determine the discriminating venue categories that distinguish each cluster.  

Note the common elements in each cluster: 

Number | Color | Common Venues
------:|-------|--------------
0 | red        | Trails  
1 | purple     | Parks and playgrounds  
2 | cyan       | Pizza and sandwich shops  
3 | sea-green  | car rentals and hotels  
4 | orange     | coffee shops

In [39]:
for cluster_num in range(5):
    cl = toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_num, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]
    display(cl.head(5))

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,"Malvern, Rouge",0,Trail,Women's Store,Farm,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Donut Shop
12,North York,Don Mills,0,Trail,Park,Other Great Outdoors,Athletics & Sports,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Women's Store
25,Scarborough,Cedarbrae,0,Construction & Landscaping,Trail,Lounge,Gaming Cafe,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market
38,North York,Bayview Village,0,Trail,Construction & Landscaping,Women's Store,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market


,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,1,Food & Drink Shop,Park,Farm,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Donut Shop
5,Etobicoke,Islington Avenue,1,Park,Baseball Field,Skating Rink,Women's Store,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market
20,York,Caledonia-Fairbanks,1,Park,Women's Store,Mexican Restaurant,Beer Store,Gym,Bakery,Sporting Goods Shop,Deli / Bodega,Food,Flower Shop
26,North York,Hillcrest Village,1,Park,Residential Building (Apartment / Condo),Dog Run,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Donut Shop
48,North York,"North Park, Maple Leaf Park, Upwood Park",1,Park,Basketball Court,Deli / Bodega,Bakery,Farmers Market,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farm


,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",2,Pizza Place,Tea Room,Chinese Restaurant,Sandwich Place,Women's Store,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School
44,North York,"York Mills, Silver Hills",2,Bank,Coffee Shop,Shopping Mall,Sandwich Place,Butcher,Baseball Field,Ice Cream Shop,Pharmacy,Burger Joint,Cosmetics Shop
50,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",2,Sandwich Place,Pharmacy,Bistro,Bank,Coffee Shop,Farm,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant
68,Etobicoke,Westmount,2,Pizza Place,Sandwich Place,Chinese Restaurant,Discount Store,Coffee Shop,Intersection,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
75,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",2,Pizza Place,American Restaurant,Sandwich Place,Coffee Shop,Liquor Store,Shopping Mall,Pharmacy,Bank,Fast Food Restaurant,Farmers Market


,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
92,Etobicoke,Northwest,3,Rental Car Location,Hotel,Women's Store,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market


,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,Victoria Village,4,Park,Pizza Place,Portuguese Restaurant,Intersection,Coffee Shop,Farm,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant
2,Downtown Toronto,"Regent Park, Harbourfront",4,Coffee Shop,Pub,Café,Music Venue,Athletics & Sports,Performing Arts Venue,Theater,Mexican Restaurant,Seafood Restaurant,Food Truck
3,North York,"Lawrence Manor, Lawrence Heights",4,Clothing Store,Bookstore,Sushi Restaurant,Men's Store,Food Court,Cosmetics Shop,American Restaurant,Furniture / Home Store,Seafood Restaurant,Café
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",4,Coffee Shop,Café,Yoga Studio,Sandwich Place,Italian Restaurant,Juice Bar,Bookstore,Fried Chicken Joint,Burger Joint,Burrito Place
7,North York,Don Mills,4,Athletics & Sports,Restaurant,Burger Joint,Bank,Women's Store,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farm
